#### WEAT Score를 통한 임베딩 모델의 편향 측정하기
- 노트 요약
    - 데이터에 내재된 편향성이 모델의 결과에도 영향을 줄 수 밖에 없음
    - 이러한 문제를 Word Embeddiing Asociation Test WEAT를 적용하여 판별
- 핵심 개념
    - 지정한 개념을 대표하는 단어들로 이루어진 단어 셋을 생성하고 해당 단어 셋한 속한 단어 들간의 거리를 계산하는 방식을 통해 워드 임베딩 공간상에 편향을 측정
    - taget
        - 단어 셋 X 
        - 단어 셋 Y 
           - X-Y을 통한 개념 축 하나 생성
    - attribute
        - 단어 셋 A
        - 단어 셋 B 
            - A-B 셋을 통한 개념 축 하나 생성
    - 거리가 가까우면 편향성이 적고, 거리가 멀면 편향성이 것으로 볼 수 있음



1. 주어진 영화 코퍼스를 바탕으로 워드임베딩 모델을 정상적으로 만들었다.
    - 워드임베딩의 **most_similar() 메소드 결과**가 의미상 바르게 나왔다.
2. 영화 구분, 장르별로 target, attribute에 대한 대표성있는 단어 셋을 생성하였다.
    - 타당한 방법론을 통해 **중복이 잘 제거**되고 개념축을 의미적으로 잘 대표하는 단어 셋이 만들어졌다.
3. WEAT score 계산 및 시각화를 정상적으로 진행하였다.
    - **전체 영화 장르별로 예술/일반 영화에 대한 편향성 WEAT score**가 상식에 부합하는 수치로 얻어졌으며 이를 잘 시각화하였다.

#### STEP 1. 형태소 분석기를 이용하여 품사가 명사인 경우 해당 단어를 추출하기

In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [2]:
# pretrain된 임베딩 모델 불러오기
# 구글뉴스가 아닌 
import os

data_dir = '~/aiffel/weat' 
model_dir = os.path.join(data_dir, 'GoogleNews-vectors-negative300.bin')

from gensim.models import KeyedVectors

# 50만개의 단어만 활용합니다. 메모리가 충분하다면 limit 파라미터값을 생략하여 300만개를 모두 활용할 수 있습니다. 
w2v = KeyedVectors.load_word2vec_format(model_dir, binary=True, limit=500000)

/home/aiffel/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
w2v

In [4]:
#print(len(w2v.vocab))   # Gensim 3.X 버전까지는 w2v.vocab을 직접 접근할 수 있습니다. 
print(len(w2v.index_to_key))   # Gensim 4.0부터는 index_to_key를 활용해 vocab size를 알 수 있습니다. 
print(len(w2v['I']))                    # 혹은 단어를 key로 직접 vector를 얻을 수 있습니다. 
print(w2v.vectors.shape)

500000
300
(500000, 300)


In [5]:
#happy 단어 벡터 출력
w2v['happy']

array([-5.18798828e-04,  1.60156250e-01,  1.60980225e-03,  2.53906250e-02,
        9.91210938e-02, -8.59375000e-02,  3.24218750e-01, -2.17285156e-02,
        1.34765625e-01,  1.10351562e-01, -1.04980469e-01, -2.90527344e-02,
       -2.38037109e-02, -4.02832031e-02, -3.68652344e-02,  2.32421875e-01,
        3.20312500e-01,  1.01074219e-01,  5.83496094e-02, -2.91824341e-04,
       -3.29589844e-02,  2.11914062e-01,  4.32128906e-02, -8.59375000e-02,
        2.81250000e-01, -1.78222656e-02,  3.79943848e-03, -1.71875000e-01,
        2.06054688e-01, -1.85546875e-01,  3.73535156e-02, -1.21459961e-02,
        2.04101562e-01, -3.80859375e-02,  3.61328125e-02, -8.15429688e-02,
        8.44726562e-02,  9.37500000e-02,  1.44531250e-01,  7.42187500e-02,
        2.51953125e-01, -7.91015625e-02,  8.69140625e-02,  1.58691406e-02,
        1.09375000e-01, -2.23632812e-01, -5.15747070e-03,  1.68945312e-01,
       -1.36718750e-01, -2.51464844e-02, -3.85742188e-02, -1.33056641e-02,
        1.38671875e-01,  

In [6]:
#happy와 유사한 단어 출력
w2v.most_similar(positive=['happy'])

[('glad', 0.7408890724182129),
 ('pleased', 0.6632170677185059),
 ('ecstatic', 0.6626912355422974),
 ('overjoyed', 0.6599286794662476),
 ('thrilled', 0.6514049172401428),
 ('satisfied', 0.6437949538230896),
 ('proud', 0.636042058467865),
 ('delighted', 0.6272379159927368),
 ('disappointed', 0.6269949674606323),
 ('excited', 0.6247665882110596)]

In [7]:
#다른 단어 살펴보기
w2v.most_similar(positive=['family'])

[('relatives', 0.6662651896476746),
 ('familiy', 0.6517067551612854),
 ('families', 0.6252894401550293),
 ('siblings', 0.6140850186347961),
 ('friends', 0.6128395199775696),
 ('mother', 0.6065612435340881),
 ('aunt', 0.5811319351196289),
 ('grandparents', 0.5762072205543518),
 ('father', 0.5717043876647949),
 ('Family', 0.5672314763069153)]

In [8]:
# 코사인 유사도 함수 선언
def cos_sim(i, j):
    return dot(i, j.T)/(norm(i)*norm(j))

# 입력 단어, A단어 셋, B단어셋 
def s(w, A, B):
    c_a = cos_sim(w, A)
    c_b = cos_sim(w, B)
    mean_A = np.mean(c_a, axis=-1)
    mean_B = np.mean(c_b, axis=-1)
    return mean_A - mean_B#, c_a, c_b

def weat_score(X, Y, A, B):
    
    s_X = s(X, A, B)
    s_Y = s(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    return  (mean_X-mean_Y)/std_dev

# print(round(weat_score(X, Y, A, B), 3))

In [9]:
# 샘플 예시
target_A = ['science', 'technology', 'physics', 'chemistry', 'Einstein', 'NASA', 'experiment', 'astronomy']
target_B = ['poetry', 'art', 'Shakespeare', 'dance', 'literature', 'novel', 'symphony', 'drama']
attribute_X = ['brother', 'father', 'uncle', 'grandfather', 'son', 'he', 'his', 'him']
attribute_Y = ['sister', 'mother', 'aunt', 'grandmother', 'daughter', 'she', 'hers', 'her']

A = np.array([w2v[word] for word in target_A])
B = np.array([w2v[word] for word in target_B])
X = np.array([w2v[word] for word in attribute_X])
Y = np.array([w2v[word] for word in attribute_Y])

weat_score(X, Y, A, B)

1.4821917

#### STEP 2. 추출된 결과로 embedding model 만들기

In [10]:
#한국어처리 패키지 설치
#!pip install konlpy 

In [11]:
#데이터셋 다운로드
#!wget https://aiffelstaticprd.blob.core.windows.net/media/documents/synopsis.zip

In [ ]:
#형태소 분석기를 활용한 토큰화
from konlpy.tag import Okt
okt = Okt()
tokenized = []
with open(os.getenv('HOME')+'/aiffel/weat/synopsis.txt', 'r') as file:
    while True:
        line = file.readline()
        if not line: break
        words = okt.pos(line, stem=True, norm=True)
        res = []
        for w in words:
            if w[1] in ["Noun"]:      # "Adjective", "Verb" 등을 포함할 수도 있습니다.
                res.append(w[0])    # 명사일 때만 tokenized 에 저장하게 됩니다. 
        tokenized.append(res)

In [ ]:
#토큰화된 단어장 크기 확인
print(len(tokenized))

In [ ]:
#벡터화
from gensim.models import Word2Vec

# tokenized에 담긴 데이터를 가지고 나만의 Word2Vec을 생성합니다. (Gensim 4.0 기준)
model = Word2Vec(tokenized, vector_size=100, window=5, min_count=3, sg=0)  
model.wv.most_similar(positive=['영화'])

# Gensim 3.X 에서는 아래와 같이 생성합니다. 
# model = Word2Vec(tokenized, size=100, window=5, min_count=3, sg=0)  
# model.most_similar(positive=['영화'])

In [ ]:
#word2vec 출력 예시
model.wv.most_similar(positive=['희망'])

#### STEP 3. target, attribute 단어 셋 만들기
    - TF-IDF 가 높은 단어임에도 불구하고 중복되는 단어가 발생
    - 개념축을 표현하는 단어 선정에 유의할것 (WEAT 계산 결과에 악영향을 미침)

#### 영화 구분
    synopsis_art.txt : 예술영화
    synopsis_gen.txt : 일반영화(상업영화)
    그 외 독립영화 등으로 분류됩니다.

#### 장르 구분
    synopsis_SF.txt: SF
    synopsis_가족.txt: 가족
    synopsis_공연.txt: 공연
    synopsis_공포(호러).txt: 공포(호러)
    synopsis_기타.txt: 기타
    synopsis_다큐멘터리.txt: 다큐멘터리
    synopsis_드라마.txt: 드라마
    synopsis_멜로로맨스.txt: 멜로로맨스
    synopsis_뮤지컬.txt: 뮤지컬
    synopsis_미스터리.txt: 미스터리
    synopsis_범죄.txt: 범죄
    synopsis_사극.txt: 사극
    synopsis_서부극(웨스턴).txt: 서부극(웨스턴)
    synopsis_성인물(에로).txt: 성인물(에로)
    synopsis_스릴러.txt: 스릴러
    synopsis_애니메이션.txt: 애니메이션
    synopsis_액션.txt: 액션
    synopsis_어드벤처.txt: 어드벤처
    synopsis_전쟁.txt: 전쟁
    synopsis_코미디.txt: 코미디
    synopsis_판타지.txt: 판타지

In [ ]:
#tf-idf 를 활용한 단어 셋 만들기
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from konlpy.tag import Okt

art_txt = 'synopsis_art.txt'
gen_txt = 'synopsis_gen.txt'

def read_token(file_name):
    okt = Okt()
    result = []
    with open(os.getenv('HOME')+'/aiffel/weat/'+file_name, 'r') as fread: 
        print(file_name, '파일을 읽고 있습니다.')
        while True:
            line = fread.readline() 
            if not line: break 
            tokenlist = okt.pos(line, stem=True, norm=True) 
            for word in tokenlist:
                if word[1] in ["Noun"]:#, "Adjective", "Verb"]:
                    result.append((word[0])) 
    return ' '.join(result)

In [ ]:
# 2개의 파일을 처리하는데 10분 가량 걸립니다. 
art = read_token(art_txt)
gen = read_token(gen_txt)

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([art, gen])

print(X.shape)

In [ ]:
print(vectorizer.vocabulary_['영화'])
print(vectorizer.get_feature_names()[23976])

In [ ]:
m1 = X[0].tocoo()   # art를 TF-IDF로 표현한 spart matrix를 가져옵니다. 
m2 = X[1].tocoo()   # gen을 TF-IDF로 표현한 spart matrix를 가져옵니다. 

w1 = [[i, j] for i, j in zip(m1.col, m1.data)]
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True)   #art를 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 
w2.sort(key=lambda x: x[1], reverse=True)   #gen을 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 

print('예술영화를 대표하는 단어들:')
for i in range(100):
    print(vectorizer.get_feature_names()[w1[i][0]], end=', ')

print('\n')
    
print('일반영화를 대표하는 단어들:')
for i in range(100):
    print(vectorizer.get_feature_names()[w2[i][0]], end=', ')

In [ ]:
#상위 100개 단어들 중 중복 단어 제외 한 상위 15개의 단어 추출
#두 개념축이 대조되려면 중복단어를 제외하여 추출하는것이 중요
n = 15
w1_, w2_ = [], []
for i in range(100):
    w1_.append(vectorizer.get_feature_names()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names()[w2[i][0]])

# w1에만 있고 w2에는 없는, 예술영화를 잘 대표하는 단어를 15개 추출한다.
target_art, target_gen = [], []
for i in range(100):
    if (w1_[i] not in w2_) and (w1_[i] in model.wv): target_art.append(w1_[i])
    if len(target_art) == n: break 

# w2에만 있고 w1에는 없는, 일반영화를 잘 대표하는 단어를 15개 추출한다.
for i in range(100):
    if (w2_[i] not in w1_) and (w2_[i] in model.wv): target_gen.append(w2_[i])
    if len(target_gen) == n: break

In [ ]:
#타겟단어 출력
print(target_art)

In [ ]:
#타겟단어 출력
print(target_gen)

In [ ]:
#장르별 대표단어 출력
#목적 -> 드라마 장르와 액션 장르를 비교하기
#목적에 해당하는 장르만 지정하기 보다 여러장르를 대표하는 단어를 선정하는 것이 더 효과적
genre_txt = ['synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_action.txt', 'synopsis_comedy.txt', 'synopsis_war.txt', 'synopsis_horror.txt']
genre_name = ['드라마', '멜로로맨스', '액션', '코미디', '전쟁', '공포(호러)']

In [ ]:
# 약 10분정도 걸립니다.
genre = []
for file_name in genre_txt:
    genre.append(read_token(file_name))

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(genre)

print(X.shape)

In [ ]:
m = [X[i].tocoo() for i in range(X.shape[0])]

w = [[[i, j] for i, j in zip(mm.col, mm.data)] for mm in m]

for i in range(len(w)):
    w[i].sort(key=lambda x: x[1], reverse=True)
attributes = []
for i in range(len(w)):
    print(genre_name[i], end=': ')
    attr = []
    j = 0
    while (len(attr) < 15):
        if vectorizer.get_feature_names()[w[i][j][0]] in model.wv:
            attr.append(vectorizer.get_feature_names()[w[i][j][0]])
            print(vectorizer.get_feature_names()[w[i][j][0]], end=', ')
        j += 1
    attributes.append(attr)
    print()

#### STEP 4. WEAT score 계산과 시각화
    - 영화 구분
    - 영화 장르
    각각 의 편향성 측정 후 WEAT score로 계산 후 Heatmap 형태로 시각화
    (attribute 구성 케이스에 대한 시각화 추가 및 강조 해보기)
    

In [ ]:
matrix = [[0 for _ in range(len(genre_name))] for _ in range(len(genre_name))]

In [ ]:
A = np.array([model.wv[word] for word in target_art])
B = np.array([model.wv[word] for word in target_gen])

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        X = np.array([model.wv[word] for word in attributes[i]])
        Y = np.array([model.wv[word] for word in attributes[j]])
        matrix[i][j] = weat_score(X, Y, A, B)

In [ ]:
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        if matrix[i][j] > 1.1 or matrix[i][j] < -1.1:
            print(genre_name[i], genre_name[j],matrix[i][j])

In [ ]:
# 모든 장르에 적용
genre_txt = ['synopsis_SF.txt', 'synopsis_family.txt', 'synopsis_show.txt', 'synopsis_horror.txt', 'synopsis_etc.txt', 
             'synopsis_documentary.txt', 'synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_musical.txt', 
             'synopsis_mystery.txt', 'synopsis_crime.txt', 'synopsis_historical.txt', 'synopsis_western.txt', 
             'synopsis_adult.txt', 'synopsis_thriller.txt', 'synopsis_animation.txt', 'synopsis_action.txt', 
             'synopsis_adventure.txt', 'synopsis_war.txt', 'synopsis_comedy.txt', 'synopsis_fantasy.txt']
genre_name = ['SF', '가족', '공연', '공포(호러)', '기타', '다큐멘터리', '드라마', '멜로로맨스', '뮤지컬', '미스터리', '범죄', '사극', '서부극(웨스턴)',
         '성인물(에로)', '스릴러', '애니메이션', '액션', '어드벤처', '전쟁', '코미디', '판타지']

In [ ]:
genre2 = []
for file_name in genre_txt:
    genre2.append(read_token(file_name))

In [ ]:
vectorizer2 = TfidfVectorizer()
X2 = vectorizer2.fit_transform(genre)

print(X2.shape)

In [ ]:
m2 = [X2[i].tocoo() for i in range(X2.shape[0])]

w2 = [[[i, j] for i, j in zip(mm2.col, mm2.data)] for mm2 in m2]

for i in range(len(w2)):
    w2[i].sort(key=lambda x: x[1], reverse=True)
attributes2 = []
for i in range(len(w2)):
    print(genre_name[i], end=': ')
    attr2 = []
    j = 0
    while (len(attr2) < 15):
        if vectorizer2.get_feature_names()[w2[i][j][0]] in model.wv:
            attr2.append(vectorizer2.get_feature_names()[w2[i][j][0]])
            print(vectorizer2.get_feature_names()[w2[i][j][0]], end=', ')
        j += 1
    attributes.append(attr2)
    print()

In [ ]:
matrix2 = [[0 for _ in range(len(genre_name))] for _ in range(len(genre_name))]

In [ ]:
A2 = np.array([model.wv[word] for word in target_art])
B2 = np.array([model.wv[word] for word in target_gen])

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        X2 = np.array([model.wv[word] for word in attributes2[i]])
        Y2 = np.array([model.wv[word] for word in attributes2[j]])
        matrix2[i][j] = weat_score(X2, Y2, A2, B2)

In [ ]:
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        if matrix2[i][j] > 1.1 or matrix2[i][j] < -1.1:
            print(genre_name[i], genre_name[j],matrix2[i][j])


In [ ]:
import numpy as np; 
import seaborn as sns; 

np.random.seed(0)

# 한글 지원 폰트
sns.set(font="Noto Sans CJK JP")

ax = sns.heatmap(matrix, xticklabels=genre_name, yticklabels=genre_name, annot=True,  cmap='RdYlGn_r')
ax

In [ ]:
np.random.seed(0)

# 한글 지원 폰트
sns.set(font="Noto Sans CJK JP")

bx = sns.heatmap(matrix2, xticklabels=genre_name, yticklabels=genre_name, annot=True,  cmap='RdYlGn_r')
bx

#### 회고
- TfidfVectorizer() 의 파라미터에 대해서 더 공부할 필요를 느낌
- 두가지 접근 방법에서 결과물의 차이를 발견하지 못함
- 좋은 데이터를 구분하고 검증하는 능력도 중요하다는 것을 알게됨
- 복잡한 모델, 큰 모델과 성능향상에 매몰되어 데이터로부터 발생될 수 있는 편향과 잘못된 결과를 발견하고 수정할 수 있으려면 연구 및 실무에서도 설명가능한 인공지능을 적용하고 활용하는것을 지향해야 할것 같다
- 한편으로는 완벽하게 중립적인 데이터, 순수한 데이터가 존재할까?그렇다면 그것이 유용하다고 할 수 있을까?
- 절대적으로 데이터가 중립성을 지켜야하는 분야나 상황, 산업은 어떤것이 있을까 더 알아보기
- 인간의 개입과 완전 자동화의 비율은 어느정도가 좋을까?
